In [ ]:
import cv2
import numpy as np

In [ ]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        x=x-10
        y=y-10
        cropped_face = img[y:y+h+50, x:x+w+50]

    return cropped_face

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      // const capture = document.createElement('button');
      // capture.textContent = 'Capture';
      // div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
!mkdir -p dataset/fahmi

In [ ]:
# Collect 100 samples of your face from webcam input
from keras.preprocessing.image import img_to_array, load_img
import imageio

count = 0
while True:
    image = take_photo()
    image = load_img(image)
    image = img_to_array(image, dtype='uint8')
    if face_extractor(image) is not None:
        count += 1
        face = cv2.resize(face_extractor(image), (400, 400))
        #face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './dataset/fahmi/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        imageio.imwrite(file_name_path, face)


        # Put count on images and display live count
        # cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        # cv2.imshow('Face Cropper', face)
        print(count)
    else:
        pass

    if count == 20: 
        break
          
print("Collecting Samples Complete")

<IPython.core.display.Javascript object>

1


<IPython.core.display.Javascript object>

2


<IPython.core.display.Javascript object>

...

18


<IPython.core.display.Javascript object>

19


<IPython.core.display.Javascript object>

20


In [ ]:
!mkdir -p dataset/empty

In [ ]:
count = 0
while True:
    image = take_photo()
    image = load_img(image)
    image = img_to_array(image, dtype='uint8')
    if image is not None:
        count += 1
        face = cv2.resize(image, (400, 400))
        #face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './dataset/empty/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        imageio.imwrite(file_name_path, face)


        # Put count on images and display live count
        # cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        # cv2.imshow('Face Cropper', face)
        print(count)
    else:
        pass

    if count == 20: 
        break
          
print("Collecting Samples Complete")

<IPython.core.display.Javascript object>

1


<IPython.core.display.Javascript object>

2


<IPython.core.display.Javascript object>

...

18


<IPython.core.display.Javascript object>

19


<IPython.core.display.Javascript object>

20


In [ ]:
!pip install split-folders tqdm

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import zipfile
import splitfolders as sf

from google.colab import files

In [ ]:
sf.ratio(input='/content/dataset/', output='/content/dataset/splitted', seed=999, ratio=(.8, .2))

Copying files: 40 files [00:00, 3113.41 files/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale = 1./255)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    "/content/dataset/splitted/train/",
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=128)

validation_generator = validation_datagen.flow_from_directory(
    "/content/dataset/splitted/val/",
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=50)

Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.summary()

model.compile(
    loss = 'categorical_crossentropy', 
    optimizer= 'adam', 
    metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

In [ ]:
history = model.fit(
    train_generator, 
    epochs=20,  
    validation_data = validation_generator, 
    verbose = 1)

Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 0.6849 - accuracy: 0.8125 - val_loss: 0.6281 - val_accuracy: 0.5000
Epoch 2/20
1/1 [==============================] - 1s 784ms/step - loss: 0.6290 - accuracy: 0.5000 - val_loss: 0.5768 - val_accuracy: 0.5000
Epoch 3/20
1/1 [==============================] - 1s 777ms/step - loss: 0.5801 - accuracy: 0.5000 - val_loss: 0.5038 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 1s 784ms/step - loss: 0.5075 - accuracy: 1.0000 - val_loss: 0.4269 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 1s 795ms/step - loss: 0.4323 - accuracy: 1.0000 - val_loss: 0.3160 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 1s 791ms/step - loss: 0.3289 - accuracy: 1.0000 - val_loss: 0.2067 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 1s 785ms/step - loss: 0.2206 - accuracy: 1.0000 - val_loss: 0.1153 - val_accuracy: 1.0000
Epoch 8/20
1/1 [=======

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takeVideo(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def take_video(label, img_data):
  data = eval_js('takeVideo("{}", "{}")'.format(label, img_data))
  return data

In [ ]:
def face_extractor_2(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

In [ ]:
import base64
from PIL import Image
import io

def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam

    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_PIL.save("temp.jpg")
    image_array = tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img("/content/temp.jpg", target_size=(150, 150, 3)))
    image_array /= 255
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.vstack([image_array])
    return image_array

In [ ]:
import requests
def postRequest(predict):
  url = "https://platform.antares.id:8443/~/antares-cse/antares-id/TrainerCV/LED1"
  payload = "{\r\n    \"m2m:cin\": {\r\n    \"con\": \"{\\\"state\\\":" + str(predict) + "}\"\r\n    }\r\n}"
  headers = {
    'x-m2m-origin': "9634da50ff7abd7a:3bdb608765b907a4",
    'content-type': "application/json;ty=4",
    'accept': "application/json",
    'cache-control': "no-cache",
    'postman-token': "e10b0cdc-98bc-4459-be34-25417d5f57bd"
    }
  response = requests.request("POST", url, data=payload, headers=headers)
  return response

In [117]:
postRequest(1)

<Response [201]>

In [ ]:
train_generator.class_indices

{'empty': 0, 'fahmi': 1}

In [119]:
import sys

start_input()
label_html = 'Capturing...'
img_data = ''
while True:
  js_reply = take_video(label_html, img_data)
  image = js_reply_to_image(js_reply)
  if image is not None:
    result = model.predict(image)
    result = np.argmax(result)
    if result == 0:
      label_html = "Empty Face Detected"
      postRequest(0)
    else :
      label_html = "Fahmi Detected"
      postRequest(1)

<IPython.core.display.Javascript object>

TypeError: ignored